# co-visitation matrix

In [1]:
import os
import sys
import gc
import subprocess
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
from tqdm import tqdm
import multiprocessing
import random
from collections import defaultdict

import pandas as pd
import cudf
import line_notify

In [2]:
SEED = 42
random.seed(SEED)

In [3]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
PREP_DIR = os.getenv("PREP_DIR")

In [4]:
CHUNK_N = 200

In [5]:
week = "week4"

In [6]:
# データ読み込み
files = [
    "train_sessions_week1.pkl",
    "test_sessions_week1.pkl",
    "train_sessions_week2.pkl",
    "test_sessions_week2.pkl",
    "train_sessions_week3.pkl",
    "test_sessions_week3.pkl",
    "train_sessions_week4.pkl",
#    "test_sessions_week4.pkl",
    "test_sessions.pkl"
]

dfs = []

for file in files:
    dfs.append(pd.read_pickle(PREP_DIR + file))
sessions = pd.concat(dfs)

# chunkのグループ分け
sessions["chunk"] = sessions["session"] % CHUNK_N

del dfs
gc.collect()

sessions["ts"] = (sessions["ts"]/1000).astype("int32")
sessions = sessions[sessions["type"]=="carts"]
sessions = sessions.drop(columns=["type"])

# chunkのグループ分け
sessions["chunk"] = sessions["session"] % CHUNK_N

In [7]:
dfs = []
for _, chunk_df in tqdm(sessions.groupby("chunk"), total=CHUNK_N):
    df = cudf.from_pandas(chunk_df)
    df = df.merge(df, on="session")
    df = df.loc[((df.ts_x - df.ts_y).abs()< 24 * 60 * 60) & (df.aid_x != df.aid_y)]
    df = df.groupby(["aid_x", "aid_y"])["session"].count().reset_index()
    df = df.rename(columns = {"session": "cnt"})
    dfs.append(df.to_pandas())

pair_df = pd.concat(dfs)
pair_df = pair_df.groupby(["aid_x", "aid_y"])["cnt"].sum().reset_index()
pair_df = pair_df.sort_values(["aid_x", "cnt"], ascending=(True, False))
pair_df = pair_df.groupby("aid_x").head(20)
pair_df.to_pickle(PREP_DIR + f"co_visitation_matrix_carts_{week}.pkl")


100%|██████████| 200/200 [00:14<00:00, 14.10it/s]


In [8]:
line_notify.send("prep_co_visitaion_matrix_carts end")